In [ ]:
!pip install -q transformers datasets
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
from datasets import load_dataset
dataset = load_dataset("LucasThil/randomized_clean_miniwob_episodes_v2")

ModuleNotFoundError: ignored

In [ ]:
dataset

NameError: ignored

In [ ]:
#drive.flush_and_unmount()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Language Tokenizer Class + Embeddings

In [ ]:
# 1 - Get out Tokenizer Class
css_fields = ['top', 'left', 'width', 'height']
special_characters = ['.', ',', '#', ':', '-', '/', '(', ')', 'https://', '@', '&', '"', "'", '!', '?', ';', '+', '=',
                      '*', '$', '€', '*', '`']


def round_to_nearest_ten(number):
    return round(number / 10) * 10

import torch.nn as nn
import torch

# Embedding Function
class EmbeddingFunction(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(EmbeddingFunction, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, input_tokens):
        embedded = self.embedding(input_tokens)
        return embedded

    # Provide a tensor and de-embbeds it to retrieve the correct index
    def get_embedding_index(self, x):
        results = torch.where(torch.sum((self.embedding.weight == x), axis=1))
        if len(results[0]) == len(x):
            return None
        else:
            return results[0][0]

# Turn it into a class
class CCNeT5Tokenizer:
    def __init__(self, vocab_path):
        stoi = {}
        itos = {}
        self.padding_char = '<PAD>'
        self.special_characters = ['.', ',', '#', ':', '-', '/', '(', ')', 'https://', '@', '&', '"', "'", '!', '?',
                                   ';', '+', '=', '*', '$', '€', '*', '`']

        with open(vocab_path, 'r') as file:
            for index, line in enumerate(file):
                line = line.strip()
                stoi[line] = index
                itos[index] = line

        stoi[' '] = stoi['']
        itos[stoi[' ']] = ' '
        self.stoi = stoi
        self.stoi[self.padding_char] = len(stoi.keys())  # Add PADDING character
        # We do not need itos as we don't implement a de-tokinezing function.
        self.itos = itos
        self.itos[len(stoi.keys()) - 1] = self.padding_char  # Add PADDING character
        print(f'pad: {len(stoi.keys()) - 1}')
        print(f'Loaded CC_NeT5 Tokenizer with vocabulary size being {len(self.stoi)}.')

        # Instantiate the embedding function
        vocab_size = len(stoi)
        self.embedding_dim = 64
        self.embedding_fn = EmbeddingFunction(vocab_size, self.embedding_dim)

    # Provide a string and tokenize it: utterance or task name.
    def tokenize_string(self, string):
        tokenized_string = []
        for w in string.lower().split(' '):
            for sc in self.special_characters:
                if sc in w:
                    w = w.replace(sc, 'Ø' + sc + 'Ø')
            p1_words = [s for s in w.split('Ø') if s != '']
            # Check if the found words exist in our vocab, else subdivide
            for fw in p1_words:
                if fw not in self.stoi.keys():
                    for c in fw:
                        tokenized_string.append(self.stoi[c])
                else:
                    tokenized_string.append(self.stoi[fw])
        return tokenized_string

    def get_tokens_from_embeddings(self, embedded_tokens):
        indices = torch.Tensor(list(map(self.embedding_fn.get_embedding_index, embedded_tokens)))
        return indices

    # Turns an array into a string
    def detokenize_array(self, array):
        reconstruted_string = []
        #pad_index = self.itos[self.stoi[self.padding_char]]
        for v in array:
            v = int(v)
            s = self.itos[v]
            if s != self.padding_char:
                reconstruted_string.append(s)

        #reconstruted_string = ' '.join(reconstruted_string)
        cleaned_string = ''
        last_single = False
        for s in reconstruted_string:
            if len(s) > 1:
                if last_single:
                    cleaned_string += ' '
                    last_single = False
                cleaned_string += s + ' '
            else:
                # We do this to treat single length characters
                cleaned_string += s
                last_single = True
        return cleaned_string

    # Truncate and pad the incoming sentences based on a max_size argument
    def truncate_pad_entry(self, tokenized_array, max_size):
        if len(tokenized_array) < max_size:
            while len(tokenized_array) < max_size:
                tokenized_array.append(self.stoi[self.padding_char])
            return tokenized_array
        elif len(tokenized_array) > max_size:
            # Too long, so we automatically truncate
            return tokenized_array[:max_size]
        else:
            # Just avoid iterating under the hood
            return tokenized_array

    # Tokenize a DOM dictionary
    def tokenize_dom(self, dom):
        tokenized_dom = []

        # Add opening tag
        element_tag = dom['tag'].lower()
        if 'input' in dom.keys():
            # if tag is input, reformat it
            element_tag = dom['input'].lower() + '_' + dom['type'].lower()
            tokenized_dom.append(self.stoi['<' + element_tag])
        else:
            # add normal tag
            tokenized_dom.append(self.stoi['<' + dom['tag'].lower()])

        for field in dom:
            if field != 'tag' and field != 'children' and field != 'type' and field != 'text' and field not in css_fields:  # and field != 'value':

                tokenized_dom.append(self.stoi[field.lower()])

                # Ensure we don't have float values, we'll stick with integers
                if isinstance(dom[field], float):
                    tokenized_dom.append(self.stoi[str(int(round_to_nearest_ten(dom[field])))])
                else:
                    words = str(dom[field]).lower().split(' ')
                    for word in words:
                        for sc in special_characters:
                            if sc in word:
                                word = word.replace(sc, 'Ø' + sc + 'Ø')
                        p1_words = [s for s in word.split('Ø') if s != '']

                        # decides whether we keep the full word, or just the letters:
                        if field == 'value' or (False and (field == 'label' or field == 'button')):
                            for w in p1_words:
                                # Take individual characters of the value string
                                processed_words = [self.stoi[w[i:i + 1]] for i in range(0, len(w), 1)]
                                tokenized_dom += processed_words

                        elif field == 'ref':
                            for w in p1_words:
                                # Take individual characters of the value string
                                processed_words = [self.stoi[w[i:i + 3]] for i in range(0, len(w), 3)]
                                tokenized_dom += processed_words
                        else:
                            # Use the full word:
                            for w in p1_words:
                                tokenized_dom.append(self.stoi[str(w)])

            elif field == 'text':
                tokenized_dom.append(self.stoi['text'])
                for c in dom[field]:
                    tokenized_dom.append(self.stoi[c])
            elif field in css_fields:
                # Cast field
                tokenized_dom.append(self.stoi[field])
                css_value = int(round_to_nearest_ten(float(dom[field])))
                tokenized_dom.append(self.stoi[str(css_value)])

        if 'children' in dom.keys():
            tokenized_dom.append(self.stoi['children'])
            for child in dom['children']:
                tokenized_dom += self.tokenize_dom(child)
                # for v in found_vocab:
                #    tokenized_dom.append(self.stoi[v])

        # add closing tag
        tokenized_dom.append(self.stoi['</' + element_tag + '>'])

        return tokenized_dom

vocab_path="/content/drive/MyDrive/WebAI/Notebooks/CC_NeT5/vocab.txt"
tokenizer = CCNeT5Tokenizer(vocab_path=vocab_path)

pad: 1590
Loaded CC_NeT5 Tokenizer with vocabulary size being 1591.


# Prepare DataLoader

In [ ]:


# Load some of the images we have here and try to save them to the dataset
import os
from typing import List, Tuple
import numpy as np
from PIL import Image
import torch
import pandas as pd
import re
import json
from torch.utils.data import DataLoader


class DatasetLoader:
    def __init__(self, screenshots_path, dataset, start_index, end_index, dom_tokenizer, batch_size):
        # Get the Subset of the dataset
        self.start_index = start_index
        self.end_index = end_index
        self.df = dataset['train'].to_pandas()[self.start_index:self.end_index]
        # When creating the class, parse JSON files into a state-based list structure
        self.df['processed_states'] = self.df['processed_states'].apply(lambda x: json.loads(re.sub(r'\b(True|False)\b', lambda m: m.group(0).lower(), x.replace("'", '"'))))
        print(f'Dataset subset has {len(self.df)} rows')

        self.screenshots_path = screenshots_path
        self.indices, found_screenshots = self.get_file_indices()
        print(f'Found a total of {found_screenshots} state screenshots for {len(self.indices)} episodes.')

        self.add_images_to_episodes()

        self.dom_tokenizer = dom_tokenizer
        self.dom_seq_length = 492
        self.utterance_seq_length = 16
        self.task_name_seq_length = 4

        self.target_action_type_seq_length = 1
        self.target_ref_seq_length = 1
        self.target_keydown_seq_length = 8 #32

        # Size of the final layer output being composed of
        # 1 (action),
        # + 64 (ref) (
        # + 64x8 (keydown text times target_keydown_seq_length)
        self.model_output_size = 577

        self.batch_size = batch_size

    # Add the images to the different episodes of the dataset
    def add_images_to_episodes(self):
        found_images = []
        i = self.start_index
        image_counter = 0
        while i < self.end_index:
            # Get K indices
            images = []
            try:
              state_indexes = self.indices[i]
              for state_index in state_indexes:
                  images.append(self.read_image(i, state_index))
                  image_counter += 1
              found_images.append(images)
            except:
              print(f'error {i}')
            i += 1
        # Add new column
        self.df['episode_images'] = found_images
        print(f'Found total of {image_counter} images in the loaded episodes.')

    def get_file_indices(self) -> List[Tuple[int, int]]:
        indices = []
        for filename in os.listdir(self.screenshots_path):
            if filename.endswith('.png'):
                # Extract the N and K values from the filename
                N, K = filename.replace('sc_', '').replace('st_', '').replace('.png', '').split('_')
                N = int(N)
                K = int(K)
                indices.append((N, K))

        # Turn the indices into a dictionary
        indices_dict = {}
        for (N, K) in indices:
            if N not in indices_dict:
                indices_dict[N] = [K]
            else:
                indices_dict[N].append(K)
                indices_dict[N] = sorted(indices_dict[N])

        return indices_dict, len(indices)

    import torch

    # Reads images into tensor
    def read_image(self, N, K) -> torch.Tensor:
        filename = f'sc_{N}_st_{K}.png'
        filepath = os.path.join(self.screenshots_path, filename)
        image = Image.open(filepath).convert('RGB')
        tensor = torch.tensor(np.array(image), dtype=torch.float32).permute(2, 0, 1) / 255.0
        return tensor

    def save_image_png(self, N, K):
        loaded_image = self.read_image(N, K).numpy()
        print(f'Image has shape {loaded_image.shape}')

        # scale the pixel values from [0,1] to [0, 255]
        array = np.clip(loaded_image * 255, 0, 255).astype('uint8')
        img = Image.fromarray(array.transpose(1, 2, 0), mode='RGB')
        img.save('screenshots/test_img.png')

    # Turn our dataset into a series of state tensors ready for training.
    # We though ensure to clearly separate our different instances:
    # - todo: T5-Data (for later, depends between SL or RL current approach)
    # - RBG
    # - todo: Tokenized DOM
    # - todo: Task Instruction
    # They are separated because they are not going to be fed in the same manner into the model.
    # Basically into one single row.
    # todo: skip rows that have negative rewards
    def process_dataset(self):
        rbg_data = []
        dom_data = []
        utterance_data = []
        task_name_data = []

        target_action = []
        target_refs = []
        target_keydown = []

        episode_previous_actions = []

        for (index, row) in self.df.iterrows():
            if float(row['reward']) <= 0: # Skips failed episodes
                continue
            if len(row['episode_images']) != len(row['processed_states']): # Skips episodes with no matching pictures amount and states
                print(f'Error: episode_images and processed_states don\' have the same length for index {index}')
                continue
            for rbg in row['episode_images']:
                rbg = np.array(rbg, dtype='float64')
                print(f'rbg shape: {rbg.shape}')
                rbg_data.append(rbg)


            # Process DOM Data
            for (index_state, state) in enumerate(row['processed_states']):

                # Output Tokens
                if state['action_type'] == 'click': # Append boolean for action click or keydown
                    t_action = np.array([0])
                    target_action.append(t_action)
                else:
                    t_action = np.array([1])
                    target_action.append(t_action)

                # Tokenize ref and appends to target
                t_ref = np.array(self.dom_tokenizer.truncate_pad_entry(self.dom_tokenizer.tokenize_string(str(state['refs'])), self.target_ref_seq_length))
                target_refs.append(t_ref)
                # Tokenize target text and appends to target
                t_keydown = np.array(self.dom_tokenizer.truncate_pad_entry(self.dom_tokenizer.tokenize_string(state['keydown_text']), self.target_keydown_seq_length))
                target_keydown.append(t_keydown)

                # Input Tokens
                # Tokenize dom
                tokenized_dom = self.dom_tokenizer.tokenize_dom(state['dom'])
                tokenized_dom = self.dom_tokenizer.truncate_pad_entry(tokenized_dom, self.dom_seq_length)
                dom_data.append(tokenized_dom)

                # Tokenize utterance
                tokenized_utterance = self.dom_tokenizer.truncate_pad_entry(self.dom_tokenizer.tokenize_string(row['utterance']), self.utterance_seq_length)
                utterance_data.append(tokenized_utterance)

                # Tokenize task name
                tokenized_task_name = self.dom_tokenizer.truncate_pad_entry(self.dom_tokenizer.tokenize_string(row['task_name']), self.task_name_seq_length)
                task_name_data.append(tokenized_task_name)

                # Deal with the previous action
                if index_state < len(row['processed_states'])-1:
                    # Default one so empty for the first action
                    if index_state == 0:
                        episode_previous_actions.append([])
                        #episode_previous_actions.append(self.dom_tokenizer.truncate_pad_entry(self.dom_tokenizer.tokenize_string(''), self.model_output_size))

                    # Add previous actions one by one
                    episode_previous_actions.append([t_action, t_ref, t_keydown])
                # Else no action to add




        rbg_data = np.stack(rbg_data)
        dom_data = np.array(dom_data)
        utterance_data = np.array(utterance_data)
        task_name_data = np.array(task_name_data)
        target_action = np.array(target_action)
        target_refs = np.array(target_refs)
        target_keydown = np.array(target_keydown)


        return rbg_data, dom_data, utterance_data, task_name_data, target_action, target_refs, target_keydown, episode_previous_actions


    # Batchify the dataset
    def get_dataloder(self, dataset):
        train_loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        return train_loader


In [ ]:
screenshots_path = '/content/drive/MyDrive/WebAI/screenshot_indexes'
batch_size = 8
dataset_loader = DatasetLoader(screenshots_path=screenshots_path, dataset=dataset, start_index=0, end_index=10, dom_tokenizer=tokenizer, batch_size=batch_size)

Dataset subset has 10 rows
Found a total of 5048 state screenshots for 2270 episodes.
Found total of 33 images in the loaded episodes.


In [ ]:
dataset_loader.indices

{9344: [0, 1],
 9345: [0, 1],
 9367: [4],
 9370: [2, 3],
 9371: [2, 3],
 9394: [2, 3],
 9395: [2, 3],
 9400: [0, 1, 2],
 9401: [0, 1, 2],
 9416: [4, 6],
 9434: [0, 2, 3],
 9435: [0, 2, 3],
 9442: [2, 3],
 9443: [2, 3],
 9448: [0, 1, 2],
 9449: [0, 1, 2],
 9455: [4],
 9461: [5, 6],
 9476: [0, 1],
 9477: [0, 1],
 9485: [4],
 9492: [0, 1],
 9493: [0, 1],
 9498: [1, 2, 3],
 9499: [1, 2, 3],
 9502: [1, 2, 3],
 9503: [1, 2, 3],
 9508: [0, 1],
 9509: [0, 1],
 9536: [0, 1, 3],
 9537: [0, 1, 3],
 9540: [0, 1, 2],
 9541: [0, 1, 2],
 9563: [4],
 9568: [5],
 9574: [0, 2, 3],
 9575: [0, 2, 3],
 9586: [4, 5],
 9587: [4, 5, 7],
 9590: [1, 2, 3],
 9591: [1, 2, 3],
 9595: [8, 9],
 9606: [5],
 9607: [5, 6, 7],
 9610: [0, 1],
 9611: [0, 1],
 9615: [9],
 9616: [10, 11],
 9620: [8, 9],
 9624: [2, 3],
 9625: [2, 3],
 9632: [4, 5, 7],
 9633: [4, 5, 7],
 9638: [6, 7],
 9639: [6, 7],
 9640: [12, 13],
 9644: [4, 5, 6],
 9645: [4],
 9652: [0, 2, 3],
 9653: [0, 2, 3],
 9654: [10],
 9656: [8, 9],
 9658: [0, 1, 3],

In [ ]:
rgb_data, dom_data, utterance_data, task_data, target_action, target_refs, target_keydown, episode_previous_actions = dataset_loader.process_dataset()

rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
Error: episode_images and processed_states don' have the same length for index 6
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)
rbg shape: (3, 210, 160)


In [ ]:
len(rgb_data), len(dom_data), len(utterance_data), len(task_data), len(target_action), len(target_refs), len(target_keydown), len(episode_previous_actions)

(14, 14, 14, 14, 14, 14, 14, 14)

In [ ]:
rgb_data.shape, dom_data.shape, utterance_data.shape, task_data.shape, target_action.shape, target_refs.shape, target_keydown.shape

((14, 3, 210, 160), (14, 492), (14, 16), (14, 4), (14, 1), (14, 1), (14, 8))

In [ ]:
import numpy as np
import torch

# Put language task together
language_input = np.concatenate((dom_data, utterance_data, task_data), axis=1)
#language_input = dom_data
print(language_input.shape)

rgb_data = torch.from_numpy(rgb_data).type(torch.float32)
language_input = torch.from_numpy(language_input).type(torch.long)
language_input = tokenizer.embedding_fn(language_input) # Create embeddings for language
print(f'rgb_data: {rgb_data.shape}, language_input: {language_input.shape}')

(14, 512)
rgb_data: torch.Size([14, 3, 210, 160]), language_input: torch.Size([14, 512, 64])


In [ ]:
language_input.shape

torch.Size([14, 512, 64])

In [ ]:
# Test formatting of previous action sequence data.
# TODO: need to update the dataloader with them
def format_previous_actions(episode_previous_actions, tokenizer, dataset_loader):
    formatted_previous_actions = []

    # Process one by one the different entries to have them correctly matching the dataset
    for entry in episode_previous_actions:
        # No previous action, have default empty sequence
        if len(entry) == 0:
            empty_action = np.array(tokenizer.truncate_pad_entry(tokenizer.tokenize_string(''), dataset_loader.model_output_size))
            print(empty_action.shape)
            formatted_previous_actions.append(empty_action)
        else:
            action_type = torch.from_numpy(entry[0]).type(torch.long)
            ref = torch.from_numpy(entry[1]).type(torch.long)
            ref = torch.flatten(tokenizer.embedding_fn(ref))
            keydown = torch.flatten(tokenizer.embedding_fn(torch.from_numpy(entry[2]).type(torch.long)))
            together = np.concatenate((action_type.detach().numpy(), ref.detach().numpy(), keydown.detach().numpy()))
            print(together.shape)
            formatted_previous_actions.append(together)

    formatted_previous_actions = np.array(formatted_previous_actions)
    return formatted_previous_actions

# These are the formatted actions to use when concatenating after the multimodal layers.
formatted_previous_actions = format_previous_actions(episode_previous_actions=episode_previous_actions, tokenizer=tokenizer, dataset_loader=dataset_loader)
print(f'Got {len(formatted_previous_actions)} formatted_previous_actions, type: {formatted_previous_actions.dtype}')

# into tensor
formatted_previous_actions = torch.from_numpy(formatted_previous_actions)
print(f'formatted_previous_actions: {formatted_previous_actions.shape}, {formatted_previous_actions.dtype}')

(577,)
(577,)
(577,)
(577,)
(577,)
(577,)
(577,)
(577,)
(577,)
(577,)
(577,)
(577,)
(577,)
(577,)
Got 14 formatted_previous_actions, type: float64
formatted_previous_actions: torch.Size([14, 577]), torch.float64


In [ ]:
def produce_labels(target_action, target_refs, target_keydown, tokenizer):
    i = 0
    labels = []

    while i<len(target_action):
        # Tokenize refs and keydowns
        embedded_action = torch.from_numpy(target_action[i]).detach().numpy()
        embedded_ref = torch.flatten(tokenizer.embedding_fn(torch.from_numpy(target_refs[i]))).detach().numpy()
        embedded_keydown = torch.flatten(tokenizer.embedding_fn(torch.from_numpy(target_keydown[i]))).detach().numpy()

        label = np.concatenate((embedded_action, embedded_ref, embedded_keydown), axis=0)
        labels.append(label)
        i += 1

    labels = np.array(labels)
    labels = torch.from_numpy(labels).type(torch.float32)
    return labels

labels = produce_labels(target_action, target_refs, target_keydown, tokenizer)
labels.shape

torch.Size([14, 577])

In [ ]:
import gc
del multimodal
gc.collect()


NameError: ignored

In [ ]:
# Batchify data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batched_rgb = dataset_loader.get_dataloder(dataset=rgb_data.to(torch.device(device)))
batched_language = dataset_loader.get_dataloder(dataset=language_input.to(torch.device(device)))
batched_previous_actions = dataset_loader.get_dataloder(dataset=formatted_previous_actions.to(torch.device(device)))
batched_labels = dataset_loader.get_dataloder(dataset=labels.to(torch.device(device)))

In [ ]:
# Some model classes

'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=2):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=(3,3), stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=(3,3),
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=14):
        super(ResNet, self).__init__()
        self.in_planes = 3

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=2)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        #self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        #out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(x)
        dl1 = out.shape
        out = self.layer2(out)
        dl2 = out.shape
        out = self.layer3(out)
        dl3 = out.shape
        out = self.layer4(out)
        dl4 = out.shape
        #out = F.avg_pool2d(out, 4)
        #out = out.view(out.size(0), -1)

        # Flatten (batch size, channels, features)
        out = out.view(8, 512, 140)
        out_1_s = out.shape
        # Might have to change the last layer to obtain our 14x11 feature vector?
        #out = self.linear(out)
        #print(f'out_1_s: {out_1_s}, out: {out.shape}')
        print(f'dl1: {dl1}, dl2: {dl2}, dl3: {dl3}, dl4: {dl4}, out: {out_1_s}')
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# ----------------------------------
# Cross Attention Model for Language
class CrossAttentionModelLanguage(nn.Module):
    def __init__(self, input_dim=64, hidden_dim=140, num_heads=4):
        super(CrossAttentionModelLanguage, self).__init__()

        self.attention = nn.MultiheadAttention(input_dim, num_heads)
        self.linear = nn.Linear(input_dim, hidden_dim)

    def forward(self, input):
        # Reshape the input to (sequence_length, batch_size, input_dim)
        input = input.permute(1, 0, 2)

        # Apply cross-attention
        output, _ = self.attention(input, input, input)

        # Reshape the output to (batch_size, sequence_length, input_dim)
        output = output.permute(1, 0, 2)

        # Apply linear transformation
        output = self.linear(output)

        return output


#--------------------------------
# Multimodal Transformer Network 
from torch.nn import Transformer

class TransformerNetwork(nn.Module):
    def __init__(self):
        super(TransformerNetwork, self).__init__()

        self.embedding_dim = 512
        self.num_layers = 8
        self.num_heads = 8

        self.embedding = nn.Linear(140, self.embedding_dim)
        self.transformer = Transformer(
            d_model=self.embedding_dim,
            nhead=self.num_heads,
            num_encoder_layers=self.num_layers,
            num_decoder_layers=self.num_layers
        )
        #self.output_layer = nn.Linear(self.embedding_dim, 1)

    def forward(self, input_tensor):
        batch_size = input_tensor.size(0)

        embedded = self.embedding(input_tensor)
        embedded = embedded.permute(1, 0, 2)

        output = self.transformer(embedded, embedded)
        output = output.permute(1, 0, 2)
        #output = self.output_layer(output)

        return output.squeeze(2)

#-------------------
# LSTM Netowkr Part
class TwoLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(TwoLSTM, self).__init__()
        self.hidden_size = hidden_size

        # Note: we use a Linear layer for dimension adjustment.
        # Indeed, the previous multi-modal transformer outputs a 1024 + prev_action length tensor,
        # and here the LSTM layers have a 512 dim where we also need to bypass residual connections.
        # We use the linear layer below to have a matching size with the input/output of these LSTMs
        # in order to concatenate them.
        self.linear = nn.Linear(input_size, hidden_size)

        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2, batch_first=True)

        # This linear layer is for the final mapping
        #self.fc = nn.Linear(hidden_size, output_size) #fully connected last layer

    def forward(self, input):
        # Perform dimension adjustment using the linear layer
        adjusted_input = self.linear(input)
        print(f'adjusted_input: {adjusted_input.shape}')

        # Perform the forward pass through each LSTM layer
        output, _hidden_state = self.lstm(adjusted_input)

        # Perform residual connections between LSTM layers
        residual_output = adjusted_input + output

        # Extract the hidden state of the last LSTM layer after residual connections
        last_hidden_state = residual_output[:, -1, :]

        #out = self.fc(last_hidden_state)
        return last_hidden_state

In [ ]:
import torch.nn as nn
import torch
#from rgb_model import ResNet18
#from language_model import CrossAttentionModelLanguage
#from multimodal_aggregate import MultiModalTransformer
#from multimodal_aggregate import TransformerNetwork
#from lstm_model import TwoLSTM
import torch.nn.functional as F

class CCNeT5(nn.Module):
    def __init__(self):
        super().__init__()
        #self.rgb_model = nn.Sequential(*resnet_blocks)
        self.rgb_model = ResNet18()
        self.language_model = CrossAttentionModelLanguage()

        # Multi Modal Part Combining RGB + Language
        #self.multimodal_transformer = MultiModalTransformer()
        self.multimodal_transformer = TransformerNetwork()

        # Here we should concatenate with the previous Action
        # when doing the forward pass

        # Two-Layer LSTM
        self.lstm = TwoLSTM(1024, 513)

        # Last output Layer to create a proper action space from the original language embeddings
        self.fc = nn.Linear(513, 577) #fully connected last layer


    # todo: pass previous action
    def forward(self, rgb_input, language_input, previous_action):
        # Process RGB input
        rgb_output = self.rgb_model(rgb_input)

        # Process language input
        language_output = self.language_model(language_input)

        print(f'shape rbg_output: {rgb_output.shape}, shape language_output: {language_output.shape}')

        # Further processing or fusion of modalities
        fused_output = torch.cat((rgb_output, language_output), dim=1)
        print(f'fused_output: {fused_output.shape} - {fused_output.dtype}')
        # Additional processing steps...

        # Permute
        #fused_output = fused_output.permute(0, 2, 1)

        # Feed into Multimodal Transformer
        multimodal_output = self.multimodal_transformer(fused_output)
        print(f'multimodal_output: {multimodal_output.shape} - type: {multimodal_output.dtype}')

        # TODO: concatenate with previous action
        print(f'previous_action: {previous_action.shape}')
        tensor2_reshaped = F.pad(previous_action, (0, 447), mode='constant')
        tensor2_expanded = torch.unsqueeze(tensor2_reshaped, dim=2)  # Shape: [8, 1024, 577]
        combined_tensor = torch.cat((multimodal_output, tensor2_expanded), dim=2).type(torch.float32) # Shape: [8, 1024, 1089]
        print(f'concatenation previous action: {combined_tensor.shape}')

        # Feed into LSTM
        # + Permute to fit the right format
        lstm_output = self.lstm(combined_tensor.permute(0, 2, 1))
        print(f'lstm_output: {lstm_output.shape}')

        # Last layer to target action space
        action_output = self.fc(lstm_output)

        return action_output

In [ ]:
multimodal = CCNeT5()
multimodal

CCNeT5(
  (rgb_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(3, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (1): BasicBlock(
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchN

In [ ]:
import torch.optim as optim
multimodal.to(device)
loss_function = nn.MSELoss()
optimizer = optim.Adam(multimodal.parameters(), lr=0.1)

def train_model(model, batched_rgb, batched_language, batched_previous_actions, batched_labels, epochs):
    i = 0
    while i < epochs:
        for rgb_data, language_data, previous_action, targets in zip(batched_rgb, batched_language, batched_previous_actions, batched_labels):
            optimizer.zero_grad()

            print(f'iteration {i}')
            if rgb_data.shape[0] != batch_size:
                # If we have a smaller batch, discard it as we don't handle it currently
                print(f'current batch size: {rgb_data.shape[0]}')
                continue
            print(rgb_data.shape)
            print(language_data.shape)
            rgb_data = rgb_data#.to(device)
            language_data = language_data#.to(device)
            action_output = model.forward(rgb_data, language_data, previous_action)
            print(f'action_output: {action_output.shape} - {action_output.dtype}')
            print(f'targets: {targets.shape} - {targets.dtype}')

            loss = loss_function(action_output, targets)
            loss.backward(retain_graph=True)
            optimizer.step()

        #if i % 100 == 0 or True:
        print("Epoch: %d, loss: %1.5f" % (i, loss.item()))

        i += 1

In [ ]:
epochs = 2
train_model(multimodal, batched_rgb, batched_language, batched_previous_actions, batched_labels, epochs)

iteration 0
torch.Size([8, 3, 210, 160])
torch.Size([8, 512, 64])
dl1: torch.Size([8, 32, 105, 80]), dl2: torch.Size([8, 128, 53, 40]), dl3: torch.Size([8, 256, 27, 20]), dl4: torch.Size([8, 512, 14, 10]), out: torch.Size([8, 512, 140])
shape rbg_output: torch.Size([8, 512, 140]), shape language_output: torch.Size([8, 512, 140])
fused_output: torch.Size([8, 1024, 140]) - torch.float32


In [ ]:
# Attempts saving model state
#torch.save(multimodal.state_dict(), 'model_save_test.pth')

# Saves the entire model
torch.save(multimodal, 'model_save_test.pth')


In [ ]:
# try to load the model and make a forward pass
model = torch.load('model_save_test.pth')
train_model(model=model, batched_rgb=batched_rgb, batched_language=batched_language)

iteration 1
torch.Size([8, 3, 210, 160])
torch.Size([8, 512, 64])
dl1: torch.Size([8, 32, 105, 80]), dl2: torch.Size([8, 128, 53, 40]), dl3: torch.Size([8, 256, 27, 20]), dl4: torch.Size([8, 512, 14, 10]), out: torch.Size([8, 512, 140])
shape rbg_output: torch.Size([8, 512, 140]), shape language_output: torch.Size([8, 512, 140])
fused_output: torch.Size([8, 1024, 140])
multimodal_output: torch.Size([8, 1024, 512])
lstm_output: torch.Size([8, 512])
iteration 2
current batch size: 6


In [ ]:
# todo:
# To have third dimension for language embeddings, use DOM + utterance + task?

In [ ]:
# todo:
# pass and concatenate previous action

In [ ]:
tokenizer.detokenize_array(tokenizer.get_tokens_from_embeddings(language_input[1]))

'<body left 0 top 0 width 1300 height 210 id classes ref 93 children <div left 0 top 0 width 160 height 210 id wrap classes ref 152 children <div left 0 top 50 width 160 height 160 id area classes ref 262 children <div left 0 top 50 width 160 height 160 id forward classes ref 369 children <div left 0 top 50 width 160 height 20 id forward - bar classes ref 33 children <span left 0 top 60 width 10 height 10 id close - forward classes ref 248 text children </span> <span left 140 top 60 width 10 height 10 id send - forward classes ref 299 text children </span> </div> <div left 0 top 70 width 160 height 40 id classes forward - header ref 160 children <div left 0 top 70 width 160 height 20 id classes forward - info ref 74 children <label left 0 top 80 width 20 height 10 id classes ref 1 text to: children </label> <input_text left 20 top 80 width 120 height 20 id classes forward - sender ref 144 value text children </input_text> </div> <div left 0 top 100 width 160 height 20 id classes forwar

In [ ]:
np.concatenate((dom_data, utterance_data, task_data), axis=1).shape

(14, 552)

In [ ]:
# Now work in creating targets of shape 577
target_action.shape, target_refs.shape, target_keydown.shape

((14, 1), (14, 1), (14, 8))

In [ ]:
print(target_action[10])

[0]


In [ ]:
'e'

'e'